In [1]:
import pandas as pd
from scipy.io import mmread

In [2]:
DATA_FOLDER = 'data/GSE161529/RAW'
RESULTS_FOLDER = 'write'

results_file = f'{RESULTS_FOLDER}/gse161529.h5ad'

## Read decompressed .mtx file

In [ ]:
mtx = mmread(f'{DATA_FOLDER}/GSM/matrix.mtx')

In [ ]:
mtx

In [ ]:
type(mtx)

In [ ]:
df = pd.DataFrame.sparse.from_spmatrix(mtx)
df

# TruncatedSDV with sparse matrix

In [ ]:
import sys
print(sys.getsizeof(df))
print(sys.getsizeof(mtx))

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
random_state = 0

In [ ]:
x = mtx.transpose()

In [ ]:
p = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),
    ('pca', TruncatedSVD(random_state=random_state))
])

In [ ]:
p.fit(x)

In [ ]:
variance = p['pca'].explained_variance_ratio_
number_of_components = np.arange(1, variance.shape[0]+1)
plt.plot(number_of_components, variance)

# PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
x = mtx.transpose().toarray()
x.shape

In [ ]:
p1 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(random_state=random_state))
])

In [ ]:
p1.fit(x)

In [ ]:
variance = p1['pca'].explained_variance_ratio_
number_of_components = np.arange(1, variance.shape[0]+1)
plt.plot(number_of_components, variance)

In [ ]:
# Gets the number of components that explain 90% of the variance
def get_number_of_components_by_variance(variance_percentages, target_variance=0.9):
    i = 0
    acc = 0
    while acc < target_variance:
        acc += variance_percentages[i]
        i += 1
    return i

n_components = get_number_of_components_by_variance(p1['pca'].explained_variance_ratio_)
print(f'Number of PCA components to be used: {n_components}')

# Test read compressed file

In [3]:
from glob import glob
from scipy.sparse import hstack, save_npz
from scipy.io import mmread

In [ ]:
DATA_FOLDER = 'data/GSE161529/RAW'

In [4]:
def read_gene_expression_matrices():
    mtx = None
    i = 1
    for filename in glob('*.mtx.gz', root_dir=DATA_FOLDER):
        print(f'Reading file {i} - {filename}')
        gene_data = mmread(f'{DATA_FOLDER}/{filename}')
        if mtx is not None:
            mtx = hstack((mtx, gene_data))
        else:
            mtx = gene_data
        i += 1
    return mtx.transpose()
mtx = read_gene_expression_matrices()

Reading file 1 - GSM4909307_ER-MH0040-matrix.mtx.gz
Reading file 2 - GSM4909271_N-MH288-Total-matrix.mtx.gz
Reading file 3 - GSM4909283_TN-SH0106-matrix.mtx.gz
Reading file 4 - GSM4909275_N-PM0372-Epi-matrix.mtx.gz
Reading file 5 - GSM4909317_ER-MH0173-T-matrix.mtx.gz
Reading file 6 - GSM4909277_B1-KCF0894-matrix.mtx.gz
Reading file 7 - GSM4909294_HER2-MH0176-matrix.mtx.gz
Reading file 8 - GSM4909309_ER-MH0043-T-matrix.mtx.gz
Reading file 9 - GSM4909297_ER-MH0125-matrix.mtx.gz
Reading file 10 - GSM4909300_ER-MH0032-matrix.mtx.gz
Reading file 11 - GSM4909290_HER2-PM0337-matrix.mtx.gz
Reading file 12 - GSM4909267_N-MH0023-Epi-matrix.mtx.gz
Reading file 13 - GSM4909261_N-PM0230-Total-matrix.mtx.gz
Reading file 14 - GSM4909292_HER2-MH0069-matrix.mtx.gz
Reading file 15 - GSM4909280_B1-MH0090-matrix.mtx.gz
Reading file 16 - GSM4909276_N-PM0372-Total-matrix.mtx.gz
Reading file 17 - GSM4909265_N-PM0233-Total-matrix.mtx.gz
Reading file 18 - GSM4909256_N-PM0095-Epi-matrix.mtx.gz
Reading file 19 

: 

: 

In [ ]:
save_npz('data.npz', mtx)